In [19]:
import pdfplumber
import re

# keywords / patterns jo hamesha skip karne hain
remove_keywords = [
    "SAVITRIBAI PHULE PUNE UNIVERSITY",
    "FORMERLY UNIVERSITY OF PUNE",
    "MASTER OF COMPUTER APPLICATIONS",
    "College Ledger",
    "PunCode :",
    "INT EXT TW",
    "Min ",
    "Max ",
    "~~~~~~",
    "MEDIUM OF INSTRUCTION",
    "continued....",
    "PUNE-411 007",
"MANAGEMENT STUDIES & RESEARCH, NASHIK"
]

text = ""
with pdfplumber.open("result.pdf") as pdf:
    # Skip first page
    for page in pdf.pages[1:]:
        page_text = page.extract_text()
        if page_text:
            for line in page_text.splitlines():
                # Page numbers or dotted lines ko regex se skip
                if re.match(r"^Page\s+\d+", line.strip()):  
                    continue
                if re.match(r"^[.]{5,}$", line.strip()):  
                    continue
                # Agar line me koi unwanted keyword hai to skip
                if any(keyword in line for keyword in remove_keywords):
                    continue
                # Agar line khali hai to skip
                if line.strip() == "":
                    continue
                text += line.strip() + "\n"

with open("output.txt", "w", encoding="utf-8") as f:
    f.write(text)

print("✅ PDF converted to output.txt successfully! (Page 1, headers, footers, page numbers, dots skipped)")


✅ PDF converted to output.txt successfully! (Page 1, headers, footers, page numbers, dots skipped)


In [21]:
first_year_records = []
second_year_records = []

with open("output.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

current_record = []
previous_line = ""

for line in lines:
    line_strip = line.strip()

    # Jab naya student start hota hai
    if line_strip.startswith("PRN:"):
        # Agar pehle ka record hai to usko classify karo
        if current_record:
            record_text = "".join(current_record)
            if "First Year " in previous_line:
                first_year_records.append(record_text)
            elif "SGPA :" in previous_line:
                second_year_records.append(record_text)
        # Naya record start
        current_record = [line]
    else:
        current_record.append(line)

    # Track karte rahenge last non-empty line ko
    if line_strip != "":
        previous_line = line_strip

# Last record bhi classify karna hoga
if current_record:
    record_text = "".join(current_record)
    if "First Year" in previous_line:
        first_year_records.append(record_text)
    elif "SGPA :" in previous_line:
        second_year_records.append(record_text)

# Save first year students
with open("first_year_students.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(first_year_records))

# Save second year students
with open("second_year_students.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(second_year_records))

print("✅ Students separated successfully!")
print(f"First Year Students: {len(first_year_records)}")
print(f"Second Year Students: {len(second_year_records)}")


✅ Students separated successfully!
First Year Students: 52
Second Year Students: 61


In [31]:
import re
import csv

fy_students = []
all_subjects = set()

with open("first_year_students.txt", "r", encoding="utf-8") as f:
    content = f.read()

# Split by PRN
students = content.split("PRN:")
students = [s.strip() for s in students if s.strip()]

for s in students:
    record = "PRN:" + s

    # Extract PRN
    prn_match = re.search(r"PRN:\s*(\d+)", record)
    prn = prn_match.group(1) if prn_match else ""

    # Extract Seat No.
    seat_match = re.search(r"SEAT NO.:\s*(\d+)", record)
    seat_no = seat_match.group(1) if seat_match else ""

    # Extract Name
    name_match = re.search(r"NAME:\s*(.+?)\s*Mother", record, re.DOTALL)
    name = name_match.group(1).replace("\n", " ").strip() if name_match else ""

    # Extract Mother's Name
    mother_match = re.search(r"Mother(?:'s)? Name\s*[:-]?\s*(.+)", record, re.DOTALL)
    if mother_match:
        mother_name = mother_match.group(1).split("\n")[0].strip()
        mother_name = mother_name.lstrip("-: ").strip()
    else:
        mother_name = ""

    # Extract SGPA
    sgpa1_match = re.search(r"First Semester SGPA\s*:\s*([0-9.]+|---)", record)
    sgpa2_match = re.search(r"Second Semester SGPA\s*:\s*([0-9.]+|---)", record)

    sgpa1 = sgpa1_match.group(1) if sgpa1_match else "---"
    sgpa2 = sgpa2_match.group(1) if sgpa2_match else "---"

    # Skip if incomplete
    if "---" in (sgpa1, sgpa2):
        continue

    sgpa1_val = float(sgpa1)
    sgpa2_val = float(sgpa2)
    cgpa = round((sgpa1_val + sgpa2_val) / 2, 2)

    # Extract subject codes with their details
    # Pattern finds subjects like 310901 or 310902C etc.
    subject_pattern = re.compile(
        r"(\d{6}[A-Z]?)\s+(?:[^\n]*?)\s+(\d{2,3})\s+\d+\s+\d+\s+([A-Z])\s+\d+\s+\d+",
        re.MULTILINE
    )

    subjects = {}
    for match in subject_pattern.finditer(record):
        sub_code = match.group(1)
        total = match.group(2)
        grade = match.group(3)
        subjects[sub_code] = {"TOTAL": total, "GRADE": grade}
        all_subjects.add(sub_code)

    fy_students.append({
        "PRN": prn,
        "Seat No.": seat_no,
        "Name": name,
        "Mother's Name": mother_name,
        "SGPA 1": sgpa1,
        "SGPA 2": sgpa2,
        "CGPA": cgpa,
        "Subjects": subjects
    })

# Create header
base_headers = ["PRN", "Seat No.", "Name", "Mother's Name", "SGPA 1", "SGPA 2", "CGPA"]
subject_headers = sorted(all_subjects)

headers = base_headers.copy()
for sub in subject_headers:
    headers.append(f"{sub}_TOTAL")
    headers.append(f"{sub}_GRADE")

# Write CSV
with open("first_year_subject_totals.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(headers)

    for st in fy_students:
        row = [
            st["PRN"],
            st["Seat No."],
            st["Name"],
            st["Mother's Name"],
            st["SGPA 1"],
            st["SGPA 2"],
            st["CGPA"]
        ]
        for sub in subject_headers:
            if sub in st["Subjects"]:
                row.append(st["Subjects"][sub]["TOTAL"])
                row.append(st["Subjects"][sub]["GRADE"])
            else:
                row.extend(["", ""])
        writer.writerow(row)

print(f"✅ CSV created: first_year_subject_totals.csv with {len(fy_students)} students and {len(subject_headers)} subjects.")


✅ CSV created: first_year_subject_totals.csv with 43 students and 18 subjects.


In [35]:

def get_top5_students(csv_file="first_year_subject_totals.csv"):
    """
    Reads the first-year CSV, sorts students by CGPA,
    and returns the top 5 students with PRN, Name, CGPA.
    """
    try:
        students = []

        with open(csv_file, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                if row["CGPA"]:  # Ensure CGPA exists
                    students.append({
                        "PRN": row["PRN"],
                        "Name": row["Name"],
                        "CGPA": float(row["CGPA"])
                    })

        # Sort descending by CGPA
        students_sorted = sorted(students, key=lambda x: x["CGPA"], reverse=True)

        # Take top 5
        top5 = students_sorted[:5]

        # Format result as list of strings
        result = []
        for idx, st in enumerate(top5, 1):
            result.append(f"{idx}. PRN: {st['PRN']}, Name: {st['Name']}, CGPA: {st['CGPA']}")

        return result  # Return list of top 5 student strings

    except Exception as e:
        return [f"❌ Error fetching top 5 students: {e}"]
print(res